### Oracle 데이터베이스

#### oracle xc 21c 설치

#### (cmd)

```
sqlplus
ID 입력
PW 입력
```


#### (cmd - sql)
데이터베이스 > 테이블스페이스 > 테이블

- 테이블스페이스 생성
```
create tablespace myts(이름)
datafile 'pydb.dbf'(물리적 파일 경로) size 10m(초기사이즈)
autoextend on 
next 10m(증가사이즈)
maxsize unlimited; 
```


- 사용자 생성하기 
시스템 계정은 관리자계정이므로, DBA(데이터베이스관리자)가 별도로 있음. 사용자 별도 생성

*oracle 12c이상인 경우
```
alter session set "_ORACLE_SCRIPT"=true;   (이름을 편하게 붙이고 싶으면 삽입)
```
```
create user python(아이디) identified by 1234(비번)
default tablespace myts(기본테이블스페이스이름);
```

- 사용자에게 권한주기
```
grant connect, resource, dba (권한) to python(아이디);
# connect DB에 접속
# resource 리소스 사용
# dba DB관리
```


#### config/setting.py
- DB를 mysql대신 오라클로 변경
```config/setting.py
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.oracle',
        'NAME': 'xe',    # DB이름
        'USER': 'python',   # 아이디
        'PASSWORD': '1234',  # 비밀번호
        'HOST': 'localhost',  # 호스트
        'PORT': '1521',   # 포트번호 (기본)
    }
}
```

#### (pycharm terminal) 
```
pip install cx_oracle
python manage.py migrate   # 위의 내용 반영
```

#### (cmd)

- 시스템계정 exit 후 일반계정으로 로그인
```
sqlplus python/1234
select * from tab;  # 테이블 목록 보여달라
```

#### (pycharm terminal)
- super user생성하기
```
python manage.py createsuperuser
사용자이름
이메일 건너뜀
pw 입력
```

#### (cmd)
```
select * from auth_user;
```

#### (oracle SQL developer)

접속 + 클릭
python 계정 저장 접속

```
select * from tab;
select * from auth_user;
```

### 3. 한줄메모장 만들기
#### (pycharm)
- 메모 앱 생성
```
python manage.py startapp memo
```

#### config/settings.py
```config/settings.py
# (제일 위에)
# 날짜 포맷 변경을 위한 모듈 로딩
from django.conf.locale.ko import formats as ko_formats
#날짜 포맷 설정
ko_formats.DATETIME_FORMAT = 'Y-m-d G:i:s'  # 4자리 연-월-일 시:분:초

# (중간에)
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'address',
    # 추가한 앱
    'memo',
```

#### memo/models.py
- 테이블을 새로 만들기 위해 models.py와 admin.py 2개의 파일을 수정해야 함
    - models.py : 테이블에 대한 모델 클래스 정의   
    - admin.py :  models.py에 등록한 테이블이 Admin 사이트에서도 보이도록 처리
- 테이블을 하나의 클래스로 정의하고 테이블의 컬럼은 클래스의 변수로 매핑
- 테이블 클래스는 django.db.models.Model 클래스를 상속받아 정의
- 변수 자료형도 장고에서 미리 정의된 자료형을 사용

```memo/models.py
# Create your models here.
from datetime import datetime
from django.db import models

# 테이블과의 매핑을 위한 클래스
class Memo(models.Model):
    # 자동증가, PK
    idx = models.AutoField(primary_key=True)
    # 필드명 = 자료형 자동증가일련번호, 기본키
    writer = models.CharField(max_length=50, blank=True, null=True)  # 이름
        # 가변사이즈문자열(최대 4000byte), blank 빈값허용 null 허용
    memo = models.CharField(max_length=2000, blank=True, null=True)
        # textfield: 대용량텍스트
    # 날짜: 현재 시각으로 자동 입력
    post_date = models.DateTimeField(default=datetime.now,\
                                     blank=True)
```

#### memo/admin.py
- admin사이트에 테이블 반영
```memo/admin.py
from django.contrib import admin
from memo.models import Memo

class MemoAdmin(admin.ModelAdmin):
    # 관리자 화면에서 관리할 필드 목록 정의
    list_display = ("writer","memo")
admin.site.register(Memo, MemoAdmin)   
```

#### (pycharm terminal)
```
python manage.py makemigrations
python manage.py migrate
```

#### (oracleSQL developer)
```
select * from tab; # 테이블 확인
desc memo_memo; # 생성된 내용 확인

IDX       NOT NULL NUMBER(11)    
WRITER             NVARCHAR2(50)   # Nvarchar2: 문자수 / varchar2: byte수
MEMO               NCLOB           # 대용량텍스트
POST_DATE NOT NULL TIMESTAMP(6)    # timestamp 1970.1.1~ ms 얼마나 경과했나

```

#### 구조
```
urls.py                   view.py                       templates
url 함수, mapping         화면준비, 데이터 처리         화면처리
```

#### memo/urls.py 생성
```memo/urls.py
from memo import views
from django.urls import path

urlpatterns = [
    # 한줄메모장 관련 url
    path('', views.home),
        # 주소, 함수
    path('insert_memo', views.insert_memo),
    path('detail', views.detail_memo),
    path('update_memo', views.update_memo),
    path('delete_memo', views.delete_memo),
]
```

#### config/urls.py
```config/urls.py
urlpatterns = [
    # 관리자용 사이트
    path('admin/', admin.site.urls),
    path('', views.home),
    path('address/', include('address.urls')),
    # 추가
    path('memo/', include('memo.urls')),
]
```

#### memo/views.py
```memo/views.py
# Create your views here.
from django.shortcuts import render, redirect
from memo.models import Memo

def home(request):
    memoList = Memo.objects.order_by("-idx")
            # Momo.objects: 테이블 전체, -필드명: 내림차순
            # = (sql) select * from momo_memo order by idx desc
    return render(request, 'memo/list.html', {'memoList': memoList, 'memoCount': len(memoList)})
            # render: 템플릿을 생성해라   # templates/memo/list.html

def insert_memo(request):
    memo = Memo(writer=request.POST['writer'], memo=request.POST['memo'])
    memo.save() # 메모 저장
            # (sql) insert into memo_memeo values (?,?)
    return redirect("/memo")
            # redirect 방향전환, 목록으로

def detail_memo(request):
    row = Memo.objects.get(idx=request.GET['idx'])
            # 상세보기
            # (sql) select * from memo_memo where idx=?
    return render(request, 'memo/detail.html', {'row': row})


def update_memo(request):
    memo = Memo( idx=request.POST['idx'],
                                 writer=request.POST['writer'],
                                 memo=request.POST['memo'] )
    memo.save()
            # (sql) update memo_memo set writer=?, memo=?  where idx=?
    return redirect("/memo")

def delete_memo(request):
    Memo.objects.get(idx=request.POST['idx']).delete()
                        # 삭제할 글번호
            # (sql) delete from memo_memo where idx=?
    return redirect("/memo")

```

#### address/templates/index.html
```address/templates/index.html
<a href="memo/">한줄메모장</a><br>  # 추가
```

#### memo 하위에 templates/memo 디렉토리 생성
#### memo/templates/memo/list.html 생성
```memo/templates/memo/list.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
    <h2>한줄메모장</h2>
    <form method="post" action="insert_memo">
        {% csrf_token %}   <!-- 보안 토큰 -->
        이름 : <input name="writer">
        메모 : <input name="memo">
        <input type="submit" value="확인">
    </form>
    <a href="/">Home</a>
    <table border="1">
        <tr>
            <th>번호</th>
            <th>이름</th>
            <th>메모</th>
            <th>날짜</th>
        </tr>
        {% for row in memoList %}    <!-- memoList가 key와 같아야 함 -->
        <!-- key -->
        <tr>
            <td>{{row.idx}}</td>   <!-- 테이블 -->
            <td>{{row.writer}}</td>
            <td><a href="detail?idx={{row.idx}}">{{row.memo}}</a></td>
            <td>{{row.post_date}}</td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>
```

#### (pycharm) 
- 웹서버 구동
```
python manage.py runserver
```
- 페이지: 관리자 로그인 후 메모 추가 

- oracle에서 확인
#### (oracle SQL)
```
select * from memo_memo;
```
페이지에서 직접 추가 후 oracle에서 확인

#### 상세페이지 추가 
#### memo/templates/memo/detail.html
```memo/templates/memo/detail.html
<!DOCTYPE html>
<html>
<head>
<script>
function update(){
    if(confirm('수정할까요?')){
        document.form1.action="update_memo"
        document.form1.submit();}
}
function del(){
    if(confirm('삭제할까요?')){   # alert물어보는 화면 추가
        document.form1.action="delete_memo"
        document.form1.submit();}

}
</script>
</head>
<body>
    <form method="post" name="form1">
        {% csrf_token %}
        <table border="1">
            <tr>
                <td>이름</td>
                <td><input name="writer" value="{{row.writer}}"></td>
            </tr>
            <tr>
                <td>날짜</td>
                <td>{{row.post_date}}</td>
            </tr>
            <tr>
                <td>메모</td>
                <td><input name="memo" value="{{row.memo}}"></td>
            </tr>
            <tr>
                <td colspan="2" align="center">
                    <input type="hidden" name="idx" value="{{row.idx}}">
                    <input type="button" value="수정" onclick="update()">
                    <input type="button" value="삭제" onclick="del()">
                </td>
            </tr>
        </table>
    </form>
</body>
</html>


```